In [ ]:
# notebook for new ircc dataset (combined_event_log_anonymous.csv)

# TODO
# √ mark subprocesses that are fully automated; incorporate in viewer
# in general, nesting subprocesses in dcr (paper)

In [ ]:
# data/combined_event_log_anonymous.csv
# -> data/combined_event_log-filt_evt1p.csv
#   -> data/combined_event_log-filt_evt1p-time1m.csv

# subprocesses:
#   data/combined_event_log-filt_evt1p-time1m.csv
#   -> data/combined_event_log-abstracted.csv + sublogs in level2 (based on status) 
#       -> data/combined_event_log-abstracted2.csv + sublogs in level1 (based on nesting)

In [1]:
import pm4py
import pandas as pd
import numpy as np
from mine_utils import get_log

# pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 200

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Preprocess logs

## Variant analysis

In [ ]:
# huge amount of variability

# total # traces = 7734 traces

# (non-abstr) new log: # vars = 7665, ratio = 99.10783553141971
#   (a variant covers at most 4 traces ...)
#   filter evts in less than X of cases:
#       1%: 7664 variants: minus 1 variant
#       10%: 7657 variants: minus 8 variants
#   same time for events within 1min:
#       ** 6030 variants: minus 1635 variants
#       + above (1%): 6014 variants: minus 1651 variants
#       + above (10%): 5841 variants: minus 1824 variants

# abstr new log: # vars = 5145, ratio = 66.5244375484872
#   same time for events within 1min: # vars = 3654, ratio = 47.25
#   + filter evts in less than 1% of cases: # vars = 3564, ratio = 46.08

# (abstr or log: # traces = 7734, # vars = 2642, ratio = 34.16084820274114)


In [ ]:
from variant_stats import get_variants_stats, get_variant_ratio, get_variant_coverage, get_covering_variants, filter_traces_on_variants

In [ ]:
log = pd.read_csv("data/combined_event_log-filt_evt1p.csv")
vars_log = get_variants_stats(log)
print(get_variant_ratio(log, vars_log))
vars_log

In [ ]:
# much bigger improvement, clearly

log = pd.read_csv("data/combined_event_log-time1m.csv")
vars_log = get_variants_stats(log)
print(get_variant_ratio(log, vars_log))
vars_log

In [ ]:
log = pd.read_csv("data/combined_event_log-filt_evt1p-time1m.csv")
vars_log = get_variants_stats(log)
print(get_variant_ratio(log, vars_log))
vars_log

In [ ]:
log = pd.read_csv("data/combined_event_log_anonymous.csv")
log['act_upd_date'] = pd.to_datetime(log['act_upd_date'])
log = log.rename({ 'app_num': 'case:concept:name', 'activity_full_value': 'concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)

abstr_log_or = pd.read_csv("data/or/abstract_log-starts_ends-v2.csv")
abstr_log_new = pd.read_csv("data/combined_event_log-abstracted2.csv")

In [ ]:
print("total # traces:", len(log['case:concept:name'].unique()))

vars_log = get_variants_stats(log)
vars_abstr_log_or = get_variants_stats(abstr_log_or)
vars_abstr_log_new = get_variants_stats(abstr_log_new)

print("log:", get_variant_ratio(log, vars_log))
print("abstr log or:", get_variant_ratio(log, vars_abstr_log_or))
print("abstr log new:", get_variant_ratio(log, vars_abstr_log_new))

## Infrequent events

In [ ]:
log = pd.read_csv("data/combined_event_log_anonymous.csv")
log = log.rename({ 'app_num': 'case:concept:name', 'activity_full_value': 'concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)

In [ ]:
log[log['activity']=='Other Reqs Assessment']['activity_status'].drop_duplicates()

In [ ]:
from log_stats import count_cases_per_event

activ_cases_counts = count_cases_per_event('concept:name', 'case:concept:name', log).reset_index()

In [ ]:
activ_cases_counts[activ_cases_counts['perc']<1] #.to_csv("data/dropped_events.csv")

In [ ]:
activ_cases_counts[activ_cases_counts['perc']<10]

In [ ]:
# let's filter on activities that occur in 1% or less of cases
to_drop = activ_cases_counts.loc[activ_cases_counts['perc']<1, 'concept:name']
log_filter = log[~ log['concept:name'].isin(to_drop)]

In [ ]:
log_filter.to_csv("data/combined_event_log-filt_evt1p.csv")

## Timestamp differences

In [ ]:
# log = pd.read_csv("data/combined_event_log_anonymous.csv")

log = pd.read_csv("data/combined_event_log-filt_evt1p.csv")
log = log.drop('Unnamed: 0', axis=1)

log = log.rename({ 'app_num': 'case:concept:name', 'activity_full_value': 'concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])

In [ ]:
from mine_utils import get_time_diff, equal_timestamps_interval

In [ ]:
log = get_time_diff(log)

In [ ]:
log[log['activity']=='Associate Medicals'].groupby('case:concept:name')['time_diff'].mean().mean()

In [ ]:
# (check all sequential events with exact same timestamp)
print("num simult (0 sec):", len(log[log['time_diff']==0]), " <> total num:", log.shape[0])

In [ ]:
# (check all sequential events with less than 1min difference in timestamps)
print("num simult (1 min):", len(log[log['time_diff'] < 60]), " <> total num:", log.shape[0])

In [ ]:
# (takes ca. 2-4 sec)

log = equal_timestamps_interval(log, 60) # 60 sec

In [ ]:
# log.to_csv("data/combined_event_log-time1m.csv")
log.to_csv("data/combined_event_log-filt_evt1p-time1m.csv")

# checkout variant analysis for seeing whether it improved matters

# Subprocesses

In [44]:
src_file = "data/combined_event_log_anonymous.csv"
# src_file = "data/combined_event_log-filt_evt1p.csv"
# src_file = "data/combined_event_log-filt_evt1p-time1m.csv"

tgt_folder = "lifecycles/all-17_10_25/data"

## Subprocesses based on status

In [65]:
# log = pd.read_csv("data/combined_event_log_anonymous.csv")
# log = log.rename({ 'app_num': 'case:concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)

log = pd.read_csv(src_file)
# log = log.drop('Unnamed: 0', axis=1)
log = log.rename(columns={"Unnamed: 0": "index"})
log

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_68041/3053757908.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv(src_file)


,index,case:concept:name,time:timestamp,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name
0,0,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Application Status,Prospective,Vietnam,Vietnam,Application Status - Prospective
1,1,0,2024-06-03 09:20:37.327,1,0,NaN,Misrepresentation,NIL,Vietnam,Vietnam,Misrepresentation - NIL
2,2,0,2024-06-03 09:20:37.327,1,0,NaN,Eligibility Assessment,NIL,Vietnam,Vietnam,Eligibility Assessment - NIL
3,3,0,2024-06-03 09:20:37.327,1,0,NaN,Final Assessment,NIL,Vietnam,Vietnam,Final Assessment - NIL
4,4,0,2024-06-03 09:20:37.327,1,0,NaN,Received,Date,Vietnam,Vietnam,Received - Date
...,...,...,...,...,...,...,...,...,...,...,...
543298,543298,7733,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,Other Reqs Assessment,NIL,Belgium,Cameroon,Other Reqs Assessment - NIL
543299,543299,7733,2024-06-26 08:11:25.043,281,1-STC2A6X,NaN,Eligibility Assessment,Review Required,Belgium,Cameroon,Eligibility Assessment - Review Required
543300,543300,7733,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Eligibility Assessment,Failed,Belgium,Cameroon,Eligibility Assessment - Failed
543301,543301,7733,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Final Assessment,Refused,Belgium,Cameroon,Final Assessment - Refused


In [66]:
# get all unique activities
log['activity'].drop_duplicates().sort_values().to_excel("data/all_activities-all.xlsx")

In [67]:
# get activity subprocesses
activ_lifecycles = log[['activity', 'activity_status']].drop_duplicates().sort_values(by=['activity'])
activ_lifecycles.to_excel("data/activity_lifecycles.xlsx")

In [68]:
# count number of sub-activities in each subprocess
counts = log[['activity', 'activity_status']].drop_duplicates().groupby('activity')['activity_status'].count()
counts = counts.sort_values(ascending=False)
counts

activity
Biographic                          16
Biometric - FCC - USA               13
Biometrics - RCMP                   12
Biometric - FCC Detail - USA         9
Entry/Exit Summary                   8
Medical Assessment                   8
Candidate ID Decision                8
Auto Promote Decision                8
Verification                         8
Biometric - FCC - NZ                 8
Candidate Search                     7
Study Permit Fee                     7
Integrated Search                    7
Eligibility Assessment               6
Biometrics Fee                       6
Candidate ID                         6
Medical                              6
Other Reqs Assessment                6
Criminality                          5
Identity Review                      5
Criminality Assessment               5
Misrepresentation                    5
Application Status                   5
VIT 34                               5
Biometric - FCC - AUS                5
Biometrics Famil

In [69]:
# only separate subprocesses with >= 2 sub-activities
parent_activ = counts[counts >= 2].reset_index()
parent_activ

,activity,activity_status
0,Biographic,16
1,Biometric - FCC - USA,13
2,Biometrics - RCMP,12
3,Biometric - FCC Detail - USA,9
4,Entry/Exit Summary,8
5,Medical Assessment,8
6,Candidate ID Decision,8
7,Auto Promote Decision,8
8,Verification,8
9,Biometric - FCC - NZ,8


In [70]:
subproc_evts = log[log['activity'].isin(parent_activ['activity'])]
subproc_evts

,index,case:concept:name,time:timestamp,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name
0,0,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Application Status,Prospective,Vietnam,Vietnam,Application Status - Prospective
1,1,0,2024-06-03 09:20:37.327,1,0,NaN,Misrepresentation,NIL,Vietnam,Vietnam,Misrepresentation - NIL
2,2,0,2024-06-03 09:20:37.327,1,0,NaN,Eligibility Assessment,NIL,Vietnam,Vietnam,Eligibility Assessment - NIL
3,3,0,2024-06-03 09:20:37.327,1,0,NaN,Final Assessment,NIL,Vietnam,Vietnam,Final Assessment - NIL
5,5,0,2024-06-03 09:20:38.890,2,1-SQ1RI5F,NaN,Biometric Assessment,Required,Vietnam,Vietnam,Biometric Assessment - Required
...,...,...,...,...,...,...,...,...,...,...,...
543298,543298,7733,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,Other Reqs Assessment,NIL,Belgium,Cameroon,Other Reqs Assessment - NIL
543299,543299,7733,2024-06-26 08:11:25.043,281,1-STC2A6X,NaN,Eligibility Assessment,Review Required,Belgium,Cameroon,Eligibility Assessment - Review Required
543300,543300,7733,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Eligibility Assessment,Failed,Belgium,Cameroon,Eligibility Assessment - Failed
543301,543301,7733,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Final Assessment,Refused,Belgium,Cameroon,Final Assessment - Refused


In [71]:
non_subproc_evts = log[~ log['activity'].isin(parent_activ['activity'])]
non_subproc_evts

,index,case:concept:name,time:timestamp,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name
4,4,0,2024-06-03 09:20:37.327,1,0,NaN,Received,Date,Vietnam,Vietnam,Received - Date
83,83,1,2024-06-03 09:20:49.697,1,0,NaN,Received,Date,Taiwan,Taiwan,Received - Date
130,130,1,2024-06-13 17:01:39.000,12,TFR163290586628,NaN,Application Transfer,Transferred,Taiwan,Taiwan,Application Transfer - Transferred
142,142,2,2024-06-03 09:20:52.143,1,0,1-SQ1QTAC,Received,Date,Mexico,Mexico,Received - Date
204,204,2,2024-07-23 14:02:17.000,16,TFR166215032639,1-SQ1QTAC,Application Transfer,Transferred,Mexico,Mexico,Application Transfer - Transferred
...,...,...,...,...,...,...,...,...,...,...,...
543166,543166,7731,2024-06-26 15:26:21.000,31,TFR164231145543,NaN,Application Transfer,Transferred,Nigeria,Nigeria,Application Transfer - Transferred
543174,543174,7732,2024-06-06 10:07:26.317,1,0,NaN,Received,Date,France,France,Received - Date
543223,543223,7732,2024-06-10 08:31:07.000,30,TFR162952326229,NaN,Application Transfer,Transferred,France,France,Application Transfer - Transferred
543224,543224,7732,2024-06-13 16:35:22.000,12,TFR163286159743,NaN,Application Transfer,Transferred,France,France,Application Transfer - Transferred


### Create separate logs per subprocess

In [72]:
from separ_subproc import separ_subproc

separ_subproc(subproc_evts, non_subproc_evts, 'activity', 'activity_status', 'concept:name', f"{tgt_folder}/level2/", f"{tgt_folder}/combined_event_log-abstracted_status.csv")

Application Status (# events: 23336)
Associate Medicals (# events: 1120)
Auto Promote (# events: 29528)
Auto Promote Decision (# events: 22315)
Biographic (# events: 30818)
Biometric - FCC - AUS (# events: 7)
Biometric - FCC - NZ (# events: 2644)
Biometric - FCC - USA (# events: 30373)
Biometric - FCC Detail - AUS (# events: 4)
Biometric - FCC Detail - USA (# events: 4764)
Biometric - FCC Reciprocal - AUS (# events: 2)
Biometric Assessment (# events: 14930)
Biometrics - RCMP (# events: 25764)
Biometrics Family Rate Fee (# events: 870)
Biometrics Fee (# events: 33173)
Candidate ID (# events: 29864)
Candidate ID Decision (# events: 24032)
Candidate Name Search (# events: 84)
Candidate Search (# events: 23552)
Compliance SP (# events: 921)
Criminality (# events: 21929)
Criminality Assessment (# events: 21927)
Derogatory Information (# events: 2)
Derogatory Verification (# events: 5)
Eligibility Assessment (# events: 19051)
Entry/Exit Detail (# events: 4)
Entry/Exit Summary (# events: 71)


,case:concept:name,activity,index,time:timestamp,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name
0,0,Application Status,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,None,Prospective,Vietnam,Vietnam,Application Status [begin]
13,0,Eligibility Assessment,2,2024-06-03 09:20:37.327,1,0,None,NIL,Vietnam,Vietnam,Eligibility Assessment [begin]
14,0,Final Assessment,3,2024-06-03 09:20:37.327,1,0,None,NIL,Vietnam,Vietnam,Final Assessment [begin]
18,0,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,None,NIL,Vietnam,Vietnam,Misrepresentation [begin]
161723,0,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,None,NIL,Vietnam,Vietnam,Misrepresentation [end]
...,...,...,...,...,...,...,...,...,...,...,...
323409,7733,Verification,543297,2024-06-17 07:46:39.197,9,1-STCSFDC,None,Completed,Belgium,Cameroon,Verification [end]
323407,7733,Other Reqs Assessment,543298,2024-06-17 07:46:39.433,281,1-STC2A6X,None,NIL,Belgium,Cameroon,Other Reqs Assessment [end]
323403,7733,Eligibility Assessment,543300,2024-06-28 10:45:09.050,281,1-STC2A6X,None,Failed,Belgium,Cameroon,Eligibility Assessment [end]
323404,7733,Final Assessment,543301,2024-06-28 10:45:09.587,281,1-STC2A6X,None,Refused,Belgium,Cameroon,Final Assessment [end]


## Subprocesses based on nesting

### Check nesting file

In [73]:
nestings = pd.read_csv("data/nested_activities-17_10_25.csv")
or_log = pd.read_csv("data/combined_event_log_anonymous.csv")

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_68041/1098523821.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  or_log = pd.read_csv("data/combined_event_log_anonymous.csv")


In [74]:
nestings[(~ nestings['Activity'].isna()) & (~nestings['Activity'].isin(or_log['activity']))]

,Parent Item,Activity


In [75]:
print(len(or_log[or_log['activity']=='Criminality Assessment']))
print(len(or_log[or_log['activity']=='Medical Assessment']))
print(len(or_log[or_log['activity']=='Misrep Assessment']))
print(len(or_log[or_log['activity']=='Security Assessment']))

21927
6675
144
41


### Find subprocesses

In [76]:
# drop na entries
nestings = nestings[~nestings['Activity'].isna()]
nestings

,Parent Item,Activity
3,Security Activity,Security Assessment
4,Security Activity,VIT 34
6,Criminality Activity,Criminality Assessment
7,Criminality Activity,Biometrics - RCMP
8,Criminality Activity,Criminality Assessment
9,Criminality Activity,Identity Review
10,Medical Activity,Medical Assessment
11,Medical Activity,Associate Medicals
12,Misrep Assessment,Misrepresentation
13,Other Reqs Assessment,Non-compliance


In [77]:
abstr_log = pd.read_csv(f"{tgt_folder}/combined_event_log-abstracted_status.csv")
abstr_log

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_68041/3098442383.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  abstr_log = pd.read_csv(f"{tgt_folder}/combined_event_log-abstracted_status.csv")


,case:concept:name,activity,index,time:timestamp,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name
0,0,Application Status,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Prospective,Vietnam,Vietnam,Application Status [begin]
1,0,Eligibility Assessment,2,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Eligibility Assessment [begin]
2,0,Final Assessment,3,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Final Assessment [begin]
3,0,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Misrepresentation [begin]
4,0,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Misrepresentation [end]
...,...,...,...,...,...,...,...,...,...,...,...
337802,7733,Verification,543297,2024-06-17 07:46:39.197,9,1-STCSFDC,NaN,Completed,Belgium,Cameroon,Verification [end]
337803,7733,Other Reqs Assessment,543298,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,NIL,Belgium,Cameroon,Other Reqs Assessment [end]
337804,7733,Eligibility Assessment,543300,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Failed,Belgium,Cameroon,Eligibility Assessment [end]
337805,7733,Final Assessment,543301,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Refused,Belgium,Cameroon,Final Assessment [end]


In [78]:
# connect parent items to events
# left merge; also keep events that are not being nested
abstr_log_parent = abstr_log.merge(nestings, left_on='activity', right_on='Activity', how='left')
abstr_log_parent

,case:concept:name,activity,index,time:timestamp,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name,Parent Item,Activity
0,0,Application Status,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Prospective,Vietnam,Vietnam,Application Status [begin],NaN,NaN
1,0,Eligibility Assessment,2,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Eligibility Assessment [begin],NaN,NaN
2,0,Final Assessment,3,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Final Assessment [begin],NaN,NaN
3,0,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Misrepresentation [begin],Misrep Assessment,Misrepresentation
4,0,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Misrepresentation [end],Misrep Assessment,Misrepresentation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
352274,7733,Verification,543297,2024-06-17 07:46:39.197,9,1-STCSFDC,NaN,Completed,Belgium,Cameroon,Verification [end],Other Reqs Assessment,Verification
352275,7733,Other Reqs Assessment,543298,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,NIL,Belgium,Cameroon,Other Reqs Assessment [end],NaN,NaN
352276,7733,Eligibility Assessment,543300,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Failed,Belgium,Cameroon,Eligibility Assessment [end],NaN,NaN
352277,7733,Final Assessment,543301,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Refused,Belgium,Cameroon,Final Assessment [end],NaN,NaN


In [79]:
subproc_evts = abstr_log_parent[abstr_log_parent['Parent Item'].notna()]
# (non-nested events; those not merged with parent)
non_subproc_evts = abstr_log_parent[abstr_log_parent['Parent Item'].isna()]

### Create separate logs per nested activity

In [80]:
from separ_subproc import separ_subproc
from shutil import copy
import os

separ_subproc(subproc_evts, non_subproc_evts, 'Parent Item', 'concept:name', 'concept:name', f"{tgt_folder}/level1", f"{tgt_folder}/combined_event_log-abstracted_nesting.csv")

Cost Recovery (# events: 34546)
Criminality Activity (# events: 43652)
Info Sharing Assessment (# events: 34720)
Medical Activity (# events: 6644)
Misrep Assessment (# events: 15468)
Other Reqs Assessment (# events: 15348)
Security Activity (# events: 42)


,case:concept:name,Parent Item,activity,index,time:timestamp,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name,Activity
4,0,Misrep Assessment,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,None,NIL,Vietnam,Vietnam,Misrep Assessment [begin],Misrepresentation
40404,0,Misrep Assessment,Misrepresentation,1,2024-06-03 09:20:37.327,1,0,None,NIL,Vietnam,Vietnam,Misrep Assessment [end],Misrepresentation
80800,0,NaN,Application Status,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Prospective,Vietnam,Vietnam,Application Status [begin],NaN
80801,0,NaN,Eligibility Assessment,2,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Eligibility Assessment [begin],NaN
80802,0,NaN,Final Assessment,3,2024-06-03 09:20:37.327,1,0,NaN,NIL,Vietnam,Vietnam,Final Assessment [begin],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80799,7733,Other Reqs Assessment,Verification,543297,2024-06-17 07:46:39.197,9,1-STCSFDC,None,Completed,Belgium,Cameroon,Other Reqs Assessment [end],Verification
282655,7733,NaN,Other Reqs Assessment,543298,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,NIL,Belgium,Cameroon,Other Reqs Assessment [end],NaN
282656,7733,NaN,Eligibility Assessment,543300,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Failed,Belgium,Cameroon,Eligibility Assessment [end],NaN
282657,7733,NaN,Final Assessment,543301,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Refused,Belgium,Cameroon,Final Assessment [end],NaN


In [81]:
if not os.path.exists(f"{tgt_folder}/level0"):
    os.mkdir(f"{tgt_folder}/level0")
copy(f"{tgt_folder}/combined_event_log-abstracted_nesting.csv", f"{tgt_folder}/level0/main.csv")

'lifecycles/all-17_10_25/data/level0/main.csv'

## Sanity check

### Check original events

In [ ]:
# same input as subprocesses part
log = pd.read_csv(src_file)
log = log.rename(columns={"Unnamed: 0": "index"})

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_68041/948523571.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv(src_file)


,index,case:concept:name,time:timestamp,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name
0,0,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Application Status,Prospective,Vietnam,Vietnam,Application Status - Prospective
1,1,0,2024-06-03 09:20:37.327,1,0,NaN,Misrepresentation,NIL,Vietnam,Vietnam,Misrepresentation - NIL
2,2,0,2024-06-03 09:20:37.327,1,0,NaN,Eligibility Assessment,NIL,Vietnam,Vietnam,Eligibility Assessment - NIL
3,3,0,2024-06-03 09:20:37.327,1,0,NaN,Final Assessment,NIL,Vietnam,Vietnam,Final Assessment - NIL
4,4,0,2024-06-03 09:20:37.327,1,0,NaN,Received,Date,Vietnam,Vietnam,Received - Date
...,...,...,...,...,...,...,...,...,...,...,...
543298,543298,7733,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,Other Reqs Assessment,NIL,Belgium,Cameroon,Other Reqs Assessment - NIL
543299,543299,7733,2024-06-26 08:11:25.043,281,1-STC2A6X,NaN,Eligibility Assessment,Review Required,Belgium,Cameroon,Eligibility Assessment - Review Required
543300,543300,7733,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Eligibility Assessment,Failed,Belgium,Cameroon,Eligibility Assessment - Failed
543301,543301,7733,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Final Assessment,Refused,Belgium,Cameroon,Final Assessment - Refused


In [ ]:
# > subprocesses based on status

from collections import defaultdict

found_indexes = set()
dir = os.path.join(tgt_folder, "level2", "logs")
sublogs_lvl2 = [ (f, pd.read_csv(os.path.join(dir, f))) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) ]

# - find original events in the sublogs (level2, status)
# (only level2 events actually exist in original log; level1 is purely for nesting)

for f, sublog in sublogs_lvl2:    
    f_name = f[0:f.index(".")]
    
    # join original log with the sublog
    log_merged = log.merge(sublog, left_on='index', right_on='index')
    # print(log_merged)
    
    # (apply same string operation as on the file name)
    activ_file = (log_merged['activity_x'].str.replace("/", "_") == f_name)
    # for all matches, activity names should correspond to file name
    assert (activ_file).all(), f_name + " <> " + log_merged[~ activ_file]['activity_x']
    # for all matches, activity statuses should correspond to sublog activity name
    assert (log_merged['activity_status'] == log_merged['concept:name_y']).all(), "file: {f_name}"
    
    found_indexes.update(log_merged['index'])

In [94]:
# - find original log events in the abstracted log (level2, status)

abstr_log = pd.read_csv(f"{tgt_folder}/combined_event_log-abstracted_status.csv")

# only interested in activities without parent (non-nested)
log_filter = log[~ log['activity'].isin(parent_activ['activity'])]
# join original log with the abstracted log
log_merged = log_filter.merge(abstr_log, left_on='index', right_on='index')
# print(log_merged)

activ_name = (log_merged['concept:name_x'] == log_merged['concept:name_y'])
# activity names should correspond
assert (activ_name).all(), log_merged[~ activ_name][['concept:name_x', 'concept:name_y']]

found_indexes.update(log_merged['index'])

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_68041/2892697805.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  abstr_log = pd.read_csv(f"{tgt_folder}/combined_event_log-abstracted_status.csv")


In [95]:
# all events found?
assert len(found_indexes) == log.shape[0], f"{len(found_indexes)} <> {log.shape[0]}"

### Check subprocesses

In [ ]:
# > subprocesses based on nesting

# per case, check if activity's start/end markers correspond to first & last sorted events of the activity

abstr_log = pd.read_csv(f"{tgt_folder}/combined_event_log-abstracted_status.csv")

def check_subproc(abstr_log, marker_label, is_start):
    # group all start/end markers by activity
    groups = abstr_log[abstr_log['concept:name'].str.endswith(marker_label)].groupby('activity')

    # for each activity & their markers for all cases
    for activ, g in groups:
        # sort on case
        g = g.sort_values(by='case:concept:name').reset_index()
        
        # find sublog corresponding to activity
        for i in [ 1, 2 ]:
            path = os.path.join(tgt_folder, f"level{i}", "logs", f"{activ.replace('/', '_')}.csv")
            if not os.path.exists(path):
                continue
            sublog = pd.read_csv(path)
        
        # print(activ, path)
        
        # group by case
        gb = sublog.groupby('case:concept:name')
        # in per-case groups, find the firsts/lasts for each case
        delims = (gb.first() if is_start else gb.last()).reset_index()
        # also sort on case
        delims = delims.sort_values(by='case:concept:name').reset_index()
        
        # firsts/lasts should be the same as the start/end markers
        assert(g['index'] == delims['index']).all(), activ
        
# for level0: check level1, level2
        
check_subproc(abstr_log, " [begin]", True)
check_subproc(abstr_log, " [end]", False)

# for level 1: check level2

path = "lifecycles/level1/logs/"

from pathlib import Path
paths = Path(path).rglob("*.csv")
for path in paths:
    abstr_sublog = pd.read_csv(path)
    check_subproc(abstr_sublog, " [begin]", True)
    check_subproc(abstr_sublog, " [end]", False)

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_68041/2435045986.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  abstr_log = pd.read_csv(f"{tgt_folder}/combined_event_log-abstracted_status.csv")


## Variant analysis (bis)

### Main variant (no filtering)

In [ ]:
from variant_stats import get_variants_stats, get_variant_ratio, get_variant_coverage, get_covering_variants, filter_traces_on_variants

In [ ]:
log = get_log("data/combined_event_log-abstracted2.csv")
log = log[['case:concept:name', 'concept:name', 'time:timestamp']]

In [ ]:
var_stats = get_variants_stats(log)
print(get_variant_ratio(log, var_stats))
var_stats

In [ ]:
singleton_vars = var_stats[var_stats['cov_amt']==1]

# ooph, a lot of traces here ...
singleton_vars['cov_amt'].sum() / var_stats['cov_amt'].sum()

### Subprocess variants (filtering!)

In [ ]:
# some ad-hoc analysis

# log = pd.read_csv("/Users/wvw/git/pm/ircc/lifecycles/level1/logs/Other Req Activity.csv")
# var_stats = get_variants_stats(log, plot=False)
# print(var_stats)
# rare_vars = var_stats[var_stats['cov_perc'] < 1]
# print()
# print(rare_vars['cov_perc'].sum().round(2))
# print((rare_vars['sequence'].count() / var_stats.shape[0] * 100).round(2))

In [ ]:
# filter the variants
# (ca. 3-5 s)

dir = "/Users/wvw/git/pm/ircc/lifecycles"
for subdir in [ "level1", "level2" ]:
    print(">", subdir)
    for path in Path(os.path.join(dir, subdir, "logs")).rglob("*.csv"):
        name = os.path.basename(path)
        log = pd.read_csv(path)
        var_stats = get_variants_stats(log, plot=False)
        
        rare_vars = var_stats[var_stats['cov_perc'] < 1]
        print(name, ":", rare_vars['cov_perc'].sum().round(2), "% traces", "(", (rare_vars['sequence'].count() / var_stats.shape[0] * 100).round(2), "% vars" ")")
        
        flog = filter_traces_on_variants(log, var_stats[var_stats['cov_perc'] >= 1])
        flog.to_csv(path)

    print()

## (Timestamp differences (bis))

In [ ]:
log = get_log("data/combined_event_log-abstracted2.csv")
log = log[['case:concept:name', 'concept:name', 'time:timestamp']]
log

In [ ]:
from mine_utils import get_time_diff

logd = get_time_diff(log)

In [ ]:
# (check all sequential events with less than 1min difference in timestamps)
print("num simult (1 min):", len(logd[ (logd['time_diff'] > 0) & (logd['time_diff'] < 60)]), " <> total num:", logd.shape[0])

# none to be found

## (Infrequent events (bis))

In [ ]:
log = pd.read_csv("data/combined_event_log-abstracted2.csv")

from log_stats import count_cases_per_event
activ_cases_counts = count_cases_per_event('concept:name', 'case:concept:name', log).reset_index()

In [ ]:
activ_cases_counts[activ_cases_counts['perc']<1]

# Mine process models

## Ad-hoc mining

In [ ]:
from variant_stats import get_variants_stats, get_variant_ratio, get_variant_coverage, get_covering_variants, filter_traces_on_variants
from mine_utils import get_log, ProcAnn, mine_heur, mine_induct, mine_alpha, mine_dfg

### Main process

In [ ]:
log = get_log("data/combined_event_log-abstracted2.csv")

cost_recov = log[(log['concept:name']=='Cost Recovery [begin]') | (log['concept:name']=='Cost Recovery [end]')]

log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
cost_recov.to_csv("data/cost_recov.csv")
xes_export.apply(cost_recov, "data/cost_recov.xes")

mine_heur(log, ProcAnn.FREQ, "graphs/combined_event_log-abstracted2-time1m")
mine_induct(log, convert_to='petri_net', output_path="graphs/combined_event_log-abstracted2-time1m-pn")

log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
xes_export.apply(log, "data/combined_event_log-abstracted2.xes")

### Subprocess

In [ ]:
verif = get_log("/Users/wvw/git/pm/ircc/lifecycles/level2/logs/Verification.csv")
verif

In [ ]:
vars = get_variants_stats(verif)
vars

In [ ]:
mine_dfg(verif)

## Systematic mining

In [ ]:
tgt_folder = "lifecycles/filt_evt1p"

In [ ]:
from mine_utils import ProcAnn, mine_heur, mine_induct, mine_alpha, mine_dfg
import pm4py.objects.log.exporter.xes.exporter as xes_export
from pathlib import Path

In [ ]:
import shutil, os

def init_subdir(subdir, subsubdirs=[]):
    if os.path.exists(subdir):
        shutil.rmtree(subdir)
    os.mkdir(subdir)
    for subsubdir in subsubdirs:
        os.mkdir(os.path.join(subdir, subsubdir))

In [ ]:
induct_formats = ['bpmn', 'petri_net']
formats_with_ann = ['dfg']

In [ ]:
def save_entries_json(names, default_format, default_ann, path):
    def entry_pref():
        if default_format in formats_with_ann:
            return f"{{ \"format\": \"{default_format}\", \"ann\": \"{default_ann.value}\" }}"
        else:
            return f"{{ \"format\": \"{default_format}\" }}"
    
    all = "[" + ", ".join(map(lambda n: f"\"{n}\"", names)) + "]"
    prefs = "{" + ", ".join(map(lambda n: f"\"{n}\": {entry_pref()}", names)) + "}"
    obj = f"{{ \"all\": {all}, \"prefs\": {prefs} }}"
    open(os.path.join(path, "graphs.json"), "w").write(obj)

In [ ]:
# mine process models for sublogs
# (ca. 1 min)

for lvl in [ 1, 2 ]:
    # subprocess level (levels 1-2)
    subdir = os.path.join(tgt_folder, f"level{lvl}")

    # default model & annotation to be shown
    default_format = "bpmn"
    default_ann = ProcAnn.FREQ

    init_subdir(os.path.join(subdir, "xes"))
    init_subdir(os.path.join(subdir, "dfg"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
    init_subdir(os.path.join(subdir, "heur"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
    for format in induct_formats:
        init_subdir(os.path.join(subdir, format))

    names = [ ]
    for path in Path(os.path.join(tgt_folder, "data", f"level{lvl}")).rglob("*.csv"):
        file = os.path.basename(path)
        name = file[0: file.index(".csv")]
        print(name)
        names.append(name)
        
        log = pd.read_csv(path)
        log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
        
        log['case:concept:name'] = log['case:concept:name'].astype('int64')
        log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])
        
        xes_export.apply(log, os.path.join(subdir, "xes", name + ".xes"))
        
        for ann in ProcAnn:
            mine_dfg(log, ann, output_path=os.path.join(subdir, "dfg", ann.value, name), save_gviz=True)
            mine_heur(log, ann, output_path=os.path.join(subdir, "heur", ann.value, name), save_gviz=True)
        
        for format in induct_formats:
            mine_induct(log, convert_to=format, output_path=os.path.join(subdir, format, name), save_gviz=True)

    save_entries_json(names, default_format, default_ann, subdir)

In [ ]:
# mine models for main process
# (ca. 30-40 sec)

log = pd.read_csv(os.path.join(tgt_folder, "data", "level0", "main.csv"))
log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])

subdir = os.path.join(tgt_folder, "level0")

default_format = "dcr"
default_ann = ProcAnn.FREQ

init_subdir(os.path.join(subdir, "logs"))
init_subdir(os.path.join(subdir, "dfg"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
init_subdir(os.path.join(subdir, "heur"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
init_subdir(os.path.join(subdir, "bpmn"))
init_subdir(os.path.join(subdir, "petri_net"))
init_subdir(os.path.join(subdir, "xes"))

name = "main"
log.to_csv(os.path.join(subdir, "logs", name + ".csv"))

for ann in ProcAnn:
    mine_dfg(log, ann, output_path=os.path.join(subdir, "dfg", ann.value, name), save_gviz=True)
    mine_heur(log, ann, output_path=os.path.join(subdir, "heur", ann.value, name), save_gviz=True)

for format in induct_formats:
    mine_induct(log, convert_to=format, output_path=os.path.join(subdir, format, name), save_gviz=True)

xes_export.apply(log, os.path.join(subdir, "xes", name + ".xes"))

save_entries_json([name], default_format, default_ann, subdir)

In [ ]:
# mine DCR: see
# /Users/wvw/git/pm/declarative/dcr4py/pm4py-dcr/ircc_dcr.ipynb